## [문제] 
- session id 부여

In [ ]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from dotenv import load_dotenv  


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()


## 3. 파일 읽기
file_name = 'C:\course_video\llm_basic\llm_0520\남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name,'r',encoding='UTF-8') as file:
    law= file.read()
    # print(law[:50])

## 4. prompt 생성    
template = [
    ('system', '''
     당신은 육아휴직 법률 전문가입니다
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
    ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5.ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
  model='gpt-4.1-nano',
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()],
)

## 6. chain 구성
chain = prompt | llm 

## 7. 대화 이력(히스토리) 관리 인스턴스 생성
history = InMemoryChatMessageHistory()

store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return history

## 9. 사용자 질문 받기
query = input("육아휴직과 관련된 질문하세요 >> ")

## 10. chain에 대화 이력(히스토리) 기능을 래필해서 추가
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
with_message_history.invoke(
    {'law': law, 'query':query},
    config={'configurable':{'session_id': 'abc'}}
    )

In [6]:
get_history('abc')

InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은 어떻게되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 기간에 대하여는 「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」 제19조(육아휴직)에 규정되어 있습니다. 육아휴직은 근로자가 신청할 경우, 임신 중인 여성 근로자 또는 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하는 근로자가 사용할 수 있으며, 그 기간은 최대 1년입니다.\n\n다만, 다음과 같은 경우에는 휴직기간이 최소 6개월이 될 수 있습니다:\n- 같은 자녀를 대상으로 부모 각각이 각각 3개월 이상 육아휴직을 사용한 경우(제19조 제2항).  \n- 법령에 따른 특별한 사유로 기간 연장이 가능하며, 2024년 개정에 따라 기간이 일부 조정될 수 있음을 참고하시기 바랍니다.\n\n또한, 육아휴직은 1년 이내로 허용되며, 근로자가 원할 경우 6개월 이상의 연장(최대 2년)도 가능합니다. 이와 관련하여, 육아휴직 기간 동안에는 해고나 불이익 조치를 할 수 없으며, 복귀 시에는 휴직 전과 같은 업무 또는 유사한 업무에 복귀시켜야 합니다(제19조 제4항).\n\n이 법률 조항은 육아휴직의 원활한 이용권 보장과 일ㆍ가정 양립 지원을 위해 마련된 것으로, 육아휴직의 구체적 사용 방법, 기간, 절차 등에 관한 세부 사항은 대통령령으로 정하고 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-75e63cbb-ce1d-48dc-a809-a46063b507c4-0')])